In [2]:
#%%

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import os, sys, gc, warnings, random

from datetime import datetime
import dateutil.relativedelta

# Data manipulation
import pandas as pd 
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, GroupKFold
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool

from tqdm.notebook import trange, tqdm

from IPython.display import display

#%% md

## Train 데이터 읽기
- 9개의 feature를 가진 780502개의 train data

In [3]:
data = pd.read_csv("../input/train.csv", parse_dates=["order_date"])
print(data.shape) # data shape 확인
data.head()

(780502, 9)


,order_id,product_id,description,quantity,order_date,price,customer_id,country,total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,11.4675,13085,United Kingdom,137.61
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,3.4650,13085,United Kingdom,166.32
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,2.0625,13085,United Kingdom,49.50


# date and times

In [4]:
date = datetime(year = 2009, month=12, day=1)
type(date)

datetime.datetime

In [5]:
from dateutil import parser
date = parser.parse("1st of December, 2009")
type(date)

datetime.datetime

In [6]:
# once you have a datetime object, 
# you can print day
date.strftime('%A')

'Tuesday'

# typed arrays of time: nparrays with dtype=datetime64

### The datetime64 dtype encodes dates as 64-bit integers.

In [7]:
date = np.array('2019-12-31', dtype=np.datetime64)
date

array('2019-12-31', dtype='datetime64[D]')

#### now we can do vertorized operations on it

In [8]:
date + np.arange(12)

array(['2019-12-31', '2020-01-01', '2020-01-02', '2020-01-03',
       '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07',
       '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11'],
      dtype='datetime64[D]')

#### For example, if you want a time resolution of one nanosecond, you only have enough information to encode a range of $2^{64}$ nanoseconds, or just under 600 years.

In [9]:
np.datetime64('2015-07-04')

numpy.datetime64('2015-07-04')

In [10]:
np.datetime64('2019-12-31 11:00')

numpy.datetime64('2019-12-31T11:00')

In [11]:
data[data['customer_id'] == 13085].groupby('order_date')['price'].describe()

,count,mean,std,min,25%,50%,75%,max
order_date,,,,,,,,
2009-12-01 07:45:00,8.0,6.734062,4.489988,2.0625,2.55750,6.64125,11.1375,11.4675
2009-12-01 07:46:00,4.0,4.331250,1.421779,2.7225,3.83625,4.20750,4.7025,6.1875
2009-12-03 14:06:00,9.0,5.949167,2.057728,2.7225,4.20750,6.18750,7.0125,9.8175
2009-12-03 14:07:00,5.0,4.438500,3.845951,2.0625,2.06250,2.72250,4.2075,11.1375
2010-01-29 10:06:00,17.0,5.760441,3.840044,0.9075,2.72250,4.86750,11.1375,11.1375
2010-01-29 11:42:00,19.0,5.440658,3.744999,0.9075,2.72250,4.86750,8.6625,11.1375
2010-10-15 15:35:00,1.0,1369.698000,NaN,1369.6980,1369.69800,1369.69800,1369.6980,1369.6980
2011-02-17 13:57:00,13.0,5.165769,4.168569,1.4025,2.06250,3.46500,8.1675,13.1175
2011-04-28 16:15:00,7.0,3.017143,1.149938,1.4025,2.22750,3.46500,3.4650,4.8675


In [144]:
total_sum_of_cus = data.groupby(['customer_id']).total.sum()
print(total_sum_of_cus.head(15))

customer_id
12346      -85.3710
12347     7749.5715
12348     3332.0100
12349     7267.4910
12350      551.7600
12351      496.5345
12352     3117.1965
12353      671.1540
12354     1781.0100
12355     1563.5565
12356    10513.3545
12357    30174.6390
12358     5286.3855
12359    14426.9235
12360     6951.2685
Name: total, dtype: float64


In [87]:
customers = total_sum_of_cus.index.to_numpy()
sums_by_cus = total_sum_of_cus.values
tiers = [chr(65+i) for i in range(5)]
tiers

['A', 'B', 'C', 'D', 'E']

In [88]:
# count, bin_dividers = np.histogram(total_sum_of_cus, bins=5)
# print(bin_dividers)

[-41433.2985 151288.7343 344010.7671 536732.7999 729454.8327 922176.8655]


In [145]:
# pd.cut 함수로 각 데이터를 5개의 bin에 할당
tier_bins = pd.qcut(x=total_sum_of_cus,               # 데이터 배열
                    q=5,
                    labels=tiers)                                        # 첫 경계값 포함
# pd.qcut

In [142]:
tier_bins = tier_bins.to_frame(name="tier")
tier_bins['customer_id'] = tier_bins.index
tier_bins = tier_bins.set_index(keys=np.arange(5914))
# tier_bins.rename(columns = {'total':'tier'})
# pd.merge(pd.DataFrame(np.arange(5914)))

In [143]:
tier_bins

,tier,customer_id
0,A,12346
1,E,12347
2,D,12348
3,E,12349
4,B,12350
...,...,...
5909,D,18283
5910,B,18284
5911,B,18285
5912,D,18286


In [58]:
total_sum_of_cus > 0

customer_id
12346    False
12347     True
12348     True
12349     True
12350     True
         ...  
18283     True
18284     True
18285     True
18286     True
18287     True
Name: total, Length: 5914, dtype: bool

In [44]:
data.groupby('year_month').total.count().size

24

In [39]:
# 고객 13085는 2010-01-29에 36개의 서로다른 물건을 구매함
data[data['customer_id'] == 13085].groupby('order_date').product_id.nunique()

order_date
2009-12-01 07:45:00     8
2009-12-01 07:46:00     4
2009-12-03 14:06:00     9
2009-12-03 14:07:00     5
2010-01-29 10:06:00    17
2010-01-29 11:42:00    19
2010-10-15 15:35:00     1
2011-02-17 13:57:00    13
2011-04-28 16:15:00     7
2011-07-05 12:11:00     9
Name: product_id, dtype: int64

In [11]:
data['customer_id'].nunique()

5914

In [20]:
data['customer_id']

0         13085
1         13085
2         13085
3         13085
4         13085
          ...  
780497    15676
780498    15676
780499    16717
780500    16717
780501    16717
Name: customer_id, Length: 780502, dtype: int64

In [140]:
# 고객등급별로 시간을 수평축, 서로 다른 상품의 개수 수직축 으로 그래프를 그리고 싶다. 산점도를 곁들여
# 고객등급별로 시간을 수평축, 시간에 따른 구매상품의 개수(sum)을 그래프로 그리고 싶다. 선 그래프를 곁들여
df = data.copy()
# data['year_month'] = data['order_date'].dt.strftime('%Y-%m')
pd.merge(df, tier_bins, on='customer_id')
# (단순) 이제 customer에 따라서 월별 팔린 상품 개수를 그래프로 그리고 싶다.

,order_id,product_id,description,quantity,order_date,price,customer_id,country,total,year_month,tier
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,11.4675,13085,United Kingdom,137.6100,2009-12,D
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.6500,2009-12,D
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.6500,2009-12,D
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,3.4650,13085,United Kingdom,166.3200,2009-12,D
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,2.0625,13085,United Kingdom,49.5000,2009-12,D
...,...,...,...,...,...,...,...,...,...,...,...
780497,579754,84997C,CHILDRENS CUTLERY POLKADOT BLUE,1,2011-11-30 14:52:00,6.8475,17911,United Kingdom,6.8475,2011-11,B
780498,579754,84997B,CHILDRENS CUTLERY RETROSPOT RED,1,2011-11-30 14:52:00,6.8475,17911,United Kingdom,6.8475,2011-11,B
780499,579754,84997A,CHILDRENS CUTLERY POLKADOT GREEN,2,2011-11-30 14:52:00,6.8475,17911,United Kingdom,13.6950,2011-11,B
780500,579754,23353,6 GIFT TAGS VINTAGE CHRISTMAS,36,2011-11-30 14:52:00,1.3695,17911,United Kingdom,49.3020,2011-11,B


## Pandas info() 함수로 데이터 타입 및 null 체크

In [16]:
#%%

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780502 entries, 0 to 780501
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   order_id     780502 non-null  object        
 1   product_id   780502 non-null  object        
 2   description  780502 non-null  object        
 3   quantity     780502 non-null  int64         
 4   order_date   780502 non-null  datetime64[ns]
 5   price        780502 non-null  float64       
 6   customer_id  780502 non-null  int64         
 7   country      780502 non-null  object        
 8   total        780502 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 53.6+ MB


#%% md

## Pandas isna(), sum() 함수로 null 데이터 개수 체크

In [17]:
#%%

data.isna().sum()

order_id       0
product_id     0
description    0
quantity       0
order_date     0
price          0
customer_id    0
country        0
total          0
dtype: int64

#%% md

## Pandas describe() 함수에 include='all' 인자 설정으로 수치형, 범주형 데이터 기본 통계량 확인

In [18]:
#%%

data.describe(include='all')

,order_id,product_id,description,quantity,order_date,price,customer_id,country,total
count,780502,780502,780502,780502.000000,780502,780502.000000,780502.000000,780502,780502.000000
unique,43955,4645,5297,NaN,40615,NaN,NaN,41,NaN
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-11-14 15:27:00,NaN,NaN,United Kingdom,NaN
freq,542,5096,5096,NaN,543,NaN,NaN,700317,NaN
first,NaN,NaN,NaN,NaN,2009-12-01 07:45:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-11-30 17:42:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,12.623307,NaN,6.140931,15313.393466,NaN,33.715373
std,NaN,NaN,NaN,143.906302,NaN,119.042266,1695.765797,NaN,274.207403
min,NaN,NaN,NaN,-74215.000000,NaN,0.000000,12346.000000,NaN,-127352.940000
25%,NaN,NaN,NaN,2.000000,NaN,2.062500,13963.000000,NaN,7.260000
